In [2]:
import json
from langchain_text_splitters import RecursiveJsonSplitter
from langchain.docstore.document import Document
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

with open('/Users/jameslee/rag/upstageailab-langchain-pjt-langchain_3/preprocessed_data/all_recipes_cleaned.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


In [3]:
documents = []
for item in data:
    # combined_text를 주 내용으로, 나머지를 메타데이터로 저장
    metadata = {
        'id': item.get('id', ''),
        'title': item.get('title', ''),
        'ingredients': item.get('ingredients', ''),
        'url': item.get('url', '')
    }
    doc = Document(page_content=item.get('combined_text', ''), metadata=metadata)
    documents.append(doc)

In [4]:
from langchain_upstage import UpstageEmbeddings

query_embeddings = UpstageEmbeddings(
    api_key=api_key,
    model="solar-embedding-1-large-query"
)

passage_embeddings = UpstageEmbeddings(
    api_key=api_key,
    model="solar-embedding-1-large-passage"
)

In [13]:
embedded_documents = passage_embeddings.embed_documents([doc.page_content for doc in documents])

In [20]:
embedded_query = query_embeddings.embed_query("피자 만드는 법을 알려줘.")
len(embedded_query)

4096

In [21]:
import numpy as np

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
similarity = np.array(embedded_query) @ np.array(embedded_documents).T

# 유사도 기준 내림차순 정렬
sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

# 결과 출력
print("냉라면 만드는 법을 알려줘.\n====================================")
cnt = 0
for i, idx in enumerate(sorted_idx):
    if cnt < 5:
        print(f"[{i}] 유사도: {similarity[idx]:.3f} | {documents[idx]}")
        print()
        cnt += 1
    else:
        break

냉라면 만드는 법을 알려줘.
[0] 유사도: 0.506 | page_content='요리 제목: 에어프라이어로 간단하게 백종원 고구마피자 만들기
필요한 재료: 고구마 중 1개, 피망 1/4개, 양파 1/4개, 베이컨 4장, 달걀 1개, 옥수수콘 1~2큰술, 소금 1/3큰술, 설탕 1큰술, 모짜렐라치즈, 파슬리가루
만드는 법: 단계 1: 고구마를 깨끗이 씻은 뒤 껍질 채로 채칼을 이용해 썰어주거나 0.3 ~ 0.4cm 두께로 두껍지 않게 썰어줍니다.도마,조리용나이프,채칼,믹싱볼 단계 2: 피망과 양파, 베이컨도 잘게 잘라 준비해 줍니다.도마,조리용나이프 단계 3: 팬에 식용유를 3큰술 정도 두르고 베이컨을 먼저 볶아줍니다.볶음팬,요리스푼고구마가 기름을 먹기 때문에 넉넉하게 둘러준 뒤 볶아주세요. 단계 4: 그 위로 고구마와 소금 1/3큰술을 넣고 골고루 볶아줍니다. 단계 5: 재료가 다 볶아지면 넓은 볼에 달걀 1개를 넣고 잘 섞어준 뒤 썰어 놓은 야채와 옥수수콘을 넣어줍니다.볼,요리젓가락 단계 6: 그 위로 볶은 베이컨과 고구마를 담은 뒤 설탕 1큰술을 넣고 골고루 잘 섞어줍니다. 단계 7: 에어프아이어 바스켓에 종이 호일을 깔고 피자 반죽을 넣어줍니다.종이호일이 때 종이 호일이 크면 공기가 잘 통할 수 있도록 피자 보다 살짝 높게 잘라주세요. 단계 8: 넣은 재료를 평평하게 잘 펼쳐준 뒤 피자치즈를 듬뿍 뿌려주고 180도에서 6~7분 정도 구워준 뒤 160도에서 5~10분 정도 더 구워줍니다.에어프라이어' metadata={'id': '7016926', 'title': '에어프라이어로 간단하게 백종원 고구마피자 만들기', 'ingredients': '고구마 중 1개, 피망 1/4개, 양파 1/4개, 베이컨 4장, 달걀 1개, 옥수수콘 1~2큰술, 소금 1/3큰술, 설탕 1큰술, 모짜렐라치즈, 파슬리가루', 'url': 'https://www.10000recipe.com/recipe/7016926'}

[1] 유사도: 0.498 | page_content='요